Importacion de librerias

In [127]:
import pandas as pd
import numpy as np
import simpy
import random
import matplotlib.pyplot as plt


Definicion de clases 

In [128]:
class Lavadora:

    def __init__(self, env, id):
        self.env = env
        self.id = id
        self.recurso = simpy.Resource(env, capacity=1) 


In [129]:
class Secadora:
    def __init__(self, env, id):
        self.env = env
        self.id = id
        self.recurso = simpy.Resource(env, capacity=1) 


In [130]:
class Lavanderia:
    def __init__(self, env, lavadoras, secadoras):
        self.env = env
        self.lavadoras = lavadoras
        self.secadoras = secadoras


In [131]:

class Cliente:
    def __init__(self, env, id, lavanderia, usa_secadora):
        self.env = env
        self.id = id
        self.lavanderia = lavanderia
        self.usa_secadora = usa_secadora  
        self.tiempo_lavado = random.randint(20, 30)  
        self.tiempo_secado = random.randint(25, 35)  
        self.eventos = pd.DataFrame()
        self.completo_servicio = False
        self.env.process(self.proceso_lavanderia())

    def registrar_evento(self, evento):
        nuevo_evento = pd.DataFrame({
            "id_cliente": [self.id],
            "evento": [evento],
            "tiempo": [self.env.now]
        })
        self.eventos = pd.concat([self.eventos, nuevo_evento])
    
    def proceso_lavanderia(self):
        tiempo_llegada = self.env.now
        self.registrar_evento("LLEGADA")
        
        print(f"{(self.env.now)}: El cliente {self.id} llegó a la lavandería")
        
        lavadoras_disponibles = []
        min_queue_length = float('inf')

        for lavadora in self.lavanderia.lavadoras:
            queue_length = len(lavadora.recurso.queue)
            if queue_length < min_queue_length:
                min_queue_length = queue_length
                lavadoras_disponibles = [lavadora]
            elif queue_length == min_queue_length:
                lavadoras_disponibles.append(lavadora)

        lavadora_seleccionada = random.choice(lavadoras_disponibles)
                
        with lavadora_seleccionada.recurso.request() as req:
            yield req  
            
            tiempo_inicio_lavado = self.env.now
            tiempo_espera_lavadora = tiempo_inicio_lavado - tiempo_llegada
            
            self.registrar_evento(f"INICIA_LAVADO_LAVADORA_{lavadora_seleccionada.id}")
            self.registrar_evento(f"TIEMPO_ESPERA_LAVADORA_{tiempo_espera_lavadora}")
            
            print(f"{(self.env.now)}: El cliente {self.id} inició lavado en la lavadora {lavadora_seleccionada.id} (tiempo: {self.tiempo_lavado} min) después de esperar {tiempo_espera_lavadora} min")
            

            yield self.env.timeout(self.tiempo_lavado)
            
            self.registrar_evento(f"FINALIZA_LAVADO_LAVADORA_{lavadora_seleccionada.id}")
            print(f"{(self.env.now)}: El cliente {self.id} terminó lavado en la lavadora {lavadora_seleccionada.id}")

        if self.usa_secadora:
            tiempo_despues_lavado = self.env.now
            
            secadoras_disponibles = []
            min_queue_length = float('inf')

            for secadora in self.lavanderia.secadoras:
                queue_length = len(secadora.recurso.queue)
                if queue_length < min_queue_length:
                    min_queue_length = queue_length
                    secadoras_disponibles = [secadora]
                elif queue_length == min_queue_length:
                    secadoras_disponibles.append(secadora)

            
            secadora_seleccionada = random.choice(secadoras_disponibles)        
            
            with secadora_seleccionada.recurso.request() as req:
                yield req  
                
                
                tiempo_inicio_secado = self.env.now
                tiempo_espera_secadora = tiempo_inicio_secado - tiempo_despues_lavado
                
                self.registrar_evento(f"INICIA_SECADO_SECADORA_{secadora_seleccionada.id}")
                self.registrar_evento(f"TIEMPO_ESPERA_SECADORA_{tiempo_espera_secadora}")
                
                print(f"{(self.env.now)}: El cliente {self.id} inició secado en la secadora {secadora_seleccionada.id} (tiempo: {self.tiempo_secado} min) después de esperar {tiempo_espera_secadora} min")
                
                yield self.env.timeout(self.tiempo_secado)
                
                self.registrar_evento(f"FINALIZA_SECADO_SECADORA_{secadora_seleccionada.id}")
                print(f"{(self.env.now)}: El cliente {self.id} terminó secado en la secadora {secadora_seleccionada.id}")
                self.registrar_evento("SERVICIO_COMPLETO")
                print(f"{(self.env.now)}: El cliente {self.id} completó todo su servicio")
                self.completo_servicio = True
        

In [132]:
class Simulacion:

    def __init__(self, env, num_lavadoras, num_secadoras):
        self.env = env
        self.lavadoras = [Lavadora(env, i+1) for i in range(num_lavadoras)]
        self.secadoras = [Secadora(env, i+1) for i in range(num_secadoras)]
        self.lavanderia = Lavanderia(env, self.lavadoras, self.secadoras)
        self.clientes = []
        self.env.process(self.generador_clientes())
    
    def generador_clientes(self):

        id_cliente = 1
        while True:
            yield self.env.timeout(random.randint(8, 12))
            
            cliente = Cliente(
                self.env,
                id_cliente,
                self.lavanderia,
                usa_secadora=random.choice([True, False])
            )
            
            self.clientes.append(cliente)
            id_cliente += 1

In [133]:
def analizar_resultados(simulacion):
    eventos = []
    for cliente in simulacion.clientes:
        eventos.append(cliente.eventos)
    
    df_eventos = pd.concat(eventos)
    df_eventos = df_eventos.reset_index()
    
    clientes_info = []
    for cliente in simulacion.clientes:
        clientes_info.append({
            "id_cliente": cliente.id,
            "tiempo_lavado": cliente.tiempo_lavado,
            "tiempo_secado": cliente.tiempo_secado,
            "usa_secadora": cliente.usa_secadora,
            "completo_servicio": cliente.completo_servicio
        })
    
    df_clientes = pd.DataFrame(clientes_info)
    

    
    return df_eventos, df_clientes

Función para ejecutar la simulación

In [134]:
def ejecutar_simulacion(duracion, num_lavadoras, num_secadoras):
    env = simpy.Environment()
    
    print("\n===== INICIANDO SIMULACIÓN DE LAVANDERÍA AUTOMÁTICA =====")
    print(f"Duración: {duracion} minutos")
    print(f"Configuración: {num_lavadoras} lavadoras, {num_secadoras} secadoras")
    print("=========================================================\n")
    
    sim = Simulacion(env, num_lavadoras, num_secadoras)
    
    env.run(until=duracion)
    
    print("\n===== SIMULACIÓN FINALIZADA =====")
    print(f"Total de clientes: {len(sim.clientes)}")
    
    df_eventos, df_clientes = analizar_resultados(sim)
    
    print("=================================\n")
    
    return sim, df_eventos, df_clientes


In [135]:
sim, df_eventos, df_clientes = ejecutar_simulacion(8*60, 3, 2)
sim


===== INICIANDO SIMULACIÓN DE LAVANDERÍA AUTOMÁTICA =====
Duración: 480 minutos
Configuración: 3 lavadoras, 2 secadoras

10: El cliente 1 llegó a la lavandería
10: El cliente 1 inició lavado en la lavadora 3 (tiempo: 25 min) después de esperar 0 min
22: El cliente 2 llegó a la lavandería
30: El cliente 3 llegó a la lavandería
30: El cliente 3 inició lavado en la lavadora 2 (tiempo: 24 min) después de esperar 0 min
35: El cliente 1 terminó lavado en la lavadora 3
35: El cliente 2 inició lavado en la lavadora 3 (tiempo: 24 min) después de esperar 13 min
38: El cliente 4 llegó a la lavandería
46: El cliente 5 llegó a la lavandería
46: El cliente 5 inició lavado en la lavadora 1 (tiempo: 20 min) después de esperar 0 min
54: El cliente 3 terminó lavado en la lavadora 2
54: El cliente 3 inició secado en la secadora 1 (tiempo: 29 min) después de esperar 0 min
58: El cliente 6 llegó a la lavandería
58: El cliente 6 inició lavado en la lavadora 2 (tiempo: 22 min) después de esperar 0 min
59: E

1- Cuanto tiempo promedio de espera para usar lavadoras las lavadoras ?


In [136]:
df_eventos, df_clientes = analizar_resultados(sim)
df = df_eventos
    
df = df.sort_values(["id_cliente", "tiempo"])

df["tiempo_anterior"] = df.groupby("id_cliente")["tiempo"].shift(1)
df["tiempo_espera"] = df["tiempo"] - df["tiempo_anterior"]

espera_lavadora = df[df["evento"].str.contains("INICIA_LAVADO")]
tiempo_espera_lavadora = espera_lavadora["tiempo"] - espera_lavadora["id_cliente"].map(
    df_eventos[df_eventos["evento"] == "LLEGADA"].set_index("id_cliente")["tiempo"]
    )
print(f"\n1. Tiempo promedio de espera para usar una lavadora: {tiempo_espera_lavadora.mean():.2f} minutos")
    


1. Tiempo promedio de espera para usar una lavadora: 7.64 minutos


2- Que porcentaje de los clientes completaron el servicio?


In [137]:
total_clientes = len(df_clientes)
clientes_completos = df_clientes["completo_servicio"].sum()
porcentaje_completos = (clientes_completos / total_clientes) * 100
print(f"\n2. Porcentaje de clientes que completaron todo el servicio: {porcentaje_completos:.2f}%")


2. Porcentaje de clientes que completaron todo el servicio: 50.00%


3 - Cual es el tiempo promedio de una persona en la lavanderia?


In [138]:
tiempo_total = []
for cliente_id in df_clientes["id_cliente"]:
    eventos_cliente = df_eventos[df_eventos["id_cliente"] == cliente_id]
    if len(eventos_cliente) >= 2:
        llegada = eventos_cliente[eventos_cliente["evento"] == "LLEGADA"]["tiempo"].iloc[0]
        ultimo_evento = eventos_cliente["tiempo"].max()
        tiempo_total.append(ultimo_evento - llegada)

if tiempo_total:
    print(f"\n3. Tiempo promedio total en la lavandería: {np.mean(tiempo_total):.2f} minutos")



3. Tiempo promedio total en la lavandería: 49.91 minutos


4 - Cuanto es el uso de las lavadoras y secadoras?


In [139]:
eventos_lavado = df_eventos[df_eventos["evento"].str.contains("INICIA_LAVADO")]
eventos_secado = df_eventos[df_eventos["evento"].str.contains("INICIA_SECADO")]

lavadoras_usadas = {}
for evento in eventos_lavado["evento"]:
    lavadora_id = int(evento.split("_")[-1])
    lavadoras_usadas[lavadora_id] = lavadoras_usadas.get(lavadora_id, 0) + 1

secadoras_usadas = {}
for evento in eventos_secado["evento"]:
    secadora_id = int(evento.split("_")[-1])
    secadoras_usadas[secadora_id] = secadoras_usadas.get(secadora_id, 0) + 1

print("\n4. Utilización de cada recurso:")
print("   Lavadoras:")
for lavadora_id, usos in sorted(lavadoras_usadas.items()):
    print(f"   - Lavadora {lavadora_id}: {usos} usos")

print("   Secadoras:")
for secadora_id, usos in sorted(secadoras_usadas.items()):
    print(f"   - Secadora {secadora_id}: {usos} usos")


4. Utilización de cada recurso:
   Lavadoras:
   - Lavadora 1: 15 usos
   - Lavadora 2: 15 usos
   - Lavadora 3: 17 usos
   Secadoras:
   - Secadora 1: 13 usos
   - Secadora 2: 13 usos


5 - Cuál es el tiempo promedio de espera para las secadoras y cómo se compara con el tiempo de espera para las lavadoras?

In [140]:
espera_secadora = df[df["evento"].str.contains("INICIA_SECADO")]
tiempo_espera_secadora = espera_secadora["tiempo"] - espera_secadora["id_cliente"].map(
    df_eventos[df_eventos["evento"].str.contains("FINALIZA_LAVADO")].set_index("id_cliente")["tiempo"]
)

print(f"\nTiempo promedio de espera para usar una secadora: {tiempo_espera_secadora.mean():.2f} minutos")
print(f"Tiempo promedio de espera para usar una lavadora: {tiempo_espera_lavadora.mean():.2f} minutos")



Tiempo promedio de espera para usar una secadora: 8.50 minutos
Tiempo promedio de espera para usar una lavadora: 7.64 minutos
